In [54]:
import mysql.connector
import stuff

In [55]:
cnx = mysql.connector.connect(
    host = stuff.host,
    user = stuff.user,
    passwd = stuff.password)

In [56]:
cursor = cnx.cursor()

In [57]:
cnx.database = 'movie_project'

In [58]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd

In [59]:
#make sure to put quotes around your url
def something(url):
    user_agent = {'User-agent': 'Mozilla/5.0'}
    page = requests.get(url, headers = user_agent)
    soup = BS(page.content, 'html.parser')
    return soup

In [60]:
#critic score stuff
def critic_func (soup):
    critic_scores=soup.find_all('div', class_='metascore_w large movie positive')
    i=0
    critic_scores_score=[]
    critic_scores_duplicate=[]
    for thing in critic_scores:
        if i%2==0:
            critic_scores_score.append(critic_scores[i].string)
        elif i%2!=0:
            critic_scores_duplicate.append(critic_scores[i].string)
        i+=1
        
    critic_scores_metacritic=[]
    for item in critic_scores_score:
        try:
            maybe=float(item)
            critic_scores_metacritic.append(maybe)
        except:
            critic_scores_metacritic.append(None)

    return critic_scores_metacritic

In [61]:
def critic_yellow_func (soup):
    critic_scores=soup.find_all('div', class_='metascore_w large movie mixed')
    i=0
    critic_scores_score=[]
    critic_scores_duplicate=[]
    for thing in critic_scores:
        if i%2==0:
            critic_scores_score.append(critic_scores[i].string)
        elif i%2!=0:
            critic_scores_duplicate.append(critic_scores[i].string)
        i+=1
        
    critic_scores_metacritic_yellow=[]
    for item in critic_scores_score:
        try:
            maybe=float(item)
            critic_scores_metacritic_yellow.append(maybe)
        except:
            critic_scores_metacritic_yellow.append(None)

    return critic_scores_metacritic_yellow

In [62]:
def critic_red_func (soup):
    critic_scores=soup.find_all('div', class_='metascore_w large movie negative')
    i=0
    critic_scores_score=[]
    critic_scores_duplicate=[]
    for thing in critic_scores:
        if i%2==0:
            critic_scores_score.append(critic_scores[i].string)
        elif i%2!=0:
            critic_scores_duplicate.append(critic_scores[i].string)
        i+=1
        
    critic_scores_metacritic_red=[]
    for item in critic_scores_score:
        try:
            maybe=float(item)
            critic_scores_metacritic_red.append(maybe)
        except:
            critic_scores_metacritic_red.append(None)

    return critic_scores_metacritic_red

In [63]:
#user score stuff
def user_func (soup):
    user_score = soup.find_all('a', class_= "metascore_anchor")
    user_score_scores=[]
    for thing in user_score:
        user_score_scores.append(thing.select('div')[0].string)
    usr_score=user_score_scores[2:301:3]
    usr_score2=[]
    for item in usr_score:
        try:
            maybe=float(item)
            usr_score2.append(maybe)
        except:
            usr_score2.append(None)
    return usr_score2


In [64]:
#titles
def title_func (soup):
    title = soup.find_all('a', class_= "title")
    titles=[]
    for thing in title:
        titles.append(thing.select('h3')[0].string)
    return titles

In [65]:
#making tuple for all data
def create_tuple_func (critic_scores_metacritic,usr_score2,titles):
    i=0
    All_data_metacritic=[]
    for thing in range(len(titles)):
        new_tuple = (titles[i],usr_score2[i],critic_scores_metacritic[i])
        All_data_metacritic.append(new_tuple)
        i+=1
    return All_data_metacritic

In [66]:
def process(url):
    soup=something(url)
    critic_scores_metacritic=critic_func(soup)
    critic_scores_metacritic_yellow=critic_yellow_func(soup)
    critic_scores_metacritic_red=critic_red_func(soup)
    critic_scores_metacritic=critic_scores_metacritic+critic_scores_metacritic_yellow+critic_scores_metacritic_red
    usr_score2=user_func(soup)
    titles=title_func(soup)
    All_data_metacritic = create_tuple_func(critic_scores_metacritic,usr_score2,titles)
    return All_data_metacritic

In [67]:
page_2=process("https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2018&sort=desc&page=1")


In [68]:
page_2

[('Have a Nice Day', 6.3, 77.0),
 ("They'll Love Me When I'm Dead", None, 77.0),
 ('On Body and Soul', 7.7, 77.0),
 ('Unlovable', None, 77.0),
 ('Monrovia, Indiana', None, 77.0),
 ("People's Republic of Desire", None, 77.0),
 ("Madeline's Madeline", 6.4, 77.0),
 ('Marlina the Murderer in Four Acts', 8.0, 77.0),
 ('The Wife', 7.3, 77.0),
 ('Leaning Into the Wind', 6.8, 76.0),
 ('A Prayer Before Dawn', 7.0, 76.0),
 ('24 Frames', 9.3, 76.0),
 ('Outside In', 7.8, 76.0),
 ('The Price of Everything', None, 76.0),
 ('Calibre', 6.5, 76.0),
 ('Ghostbox Cowboy', None, 76.0),
 ('Tito and the Birds', None, 76.0),
 ("At Eternity's Gate", 7.0, 76.0),
 ('Garry Winogrand: All Things are Photographable', None, 76.0),
 ('Under The Wire', 8.3, 75.0),
 ('Border', 6.4, 75.0),
 ('Bodied', 7.9, 75.0),
 ('Blaze', 5.6, 75.0),
 ('This Is Congo', None, 75.0),
 ('Capernaum', 8.0, 75.0),
 ('The Kindergarten Teacher', 7.6, 75.0),
 ('Stan & Ollie', 7.5, 75.0),
 ('Under the Tree', 6.0, 75.0),
 ('Jinn', 4.0, 75.0),
 (

In [69]:
insert_stm = """INSERT INTO metacritic (title, critic_score, user_score) 
VALUES (%s, %s, %s)"""
cursor.executemany(insert_stm, page_2)
cnx.commit()

In [70]:
page_3=process("https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2018&sort=desc&page=2")


In [71]:
insert_stm = """INSERT INTO metacritic (title, critic_score, user_score) 
VALUES (%s, %s, %s)"""
cursor.executemany(insert_stm, page_3)
cnx.commit()

In [73]:
page_4=process("https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2018&sort=desc&page=3")


In [74]:
insert_stm = """INSERT INTO metacritic (title, critic_score, user_score) 
VALUES (%s, %s, %s)"""
cursor.executemany(insert_stm, page_4)
cnx.commit()

In [75]:
page_5=process("https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2018&sort=desc&page=4")


In [76]:
insert_stm = """INSERT INTO metacritic (title, critic_score, user_score) 
VALUES (%s, %s, %s)"""
cursor.executemany(insert_stm, page_5)
cnx.commit()

In [77]:
page_6=process("https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2018&sort=desc&page=5")


In [78]:
insert_stm = """INSERT INTO metacritic (title, critic_score, user_score) 
VALUES (%s, %s, %s)"""
cursor.executemany(insert_stm, page_6)
cnx.commit()

In [79]:
page_7=process("https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2018&sort=desc&page=6")


In [80]:
insert_stm = """INSERT INTO metacritic (title, critic_score, user_score) 
VALUES (%s, %s, %s)"""
cursor.executemany(insert_stm, page_7)
cnx.commit()